In [1]:
import json, glob
import osgeo.ogr as ogr
import numpy as np
import rasterio as rio
import rasterio.features as features
import rasterio.warp as warp
from shapely.geometry import shape, asShape, GeometryCollection, MultiPolygon
from shapely.wkb import loads as wkb_loads

In [2]:
imagens = glob.glob('*.tif')
imagens

['mapbiomas-matogrossodosul-2018-corte.tif']

In [3]:
def open_raster(imagens):
    crs_affine_all = []
    img_crs_all = []
    geom_all = {}
    
    for img in imagens:
        
        img_open = rio.open(img)
        img_arr = img_open.read() # array das img
        crs_affine_all.append(img_open.transform) # pega as coordenadas das img
        img_crs_all.append(img_open.crs) #pega os crs das img
        
        name = img.split(".")[0]
        geom_all[name] = []
        names = [name]
        
        for geom, val in features.shapes(img_arr, transform = img_open.transform):

            geom = warp.transform_geom(img_open.crs, 'EPSG:4326', geom, precision=5)
            geom_all[name].append(geom)

    return img_open, img_arr, crs_affine_all, img_crs_all, geom_all, names

In [4]:
img_open, img_arr, crs_affine_all, img_crs, geom_all, names = open_raster(imagens)

In [ ]:
print(teste)

In [ ]:
json.dumps(geom_all['mapbiomas-matogrossodosul-2018-corte'])

In [ ]:
list(geom_all.keys())[0]

In [ ]:
def rasterio_to_shapely(names, geom_all):
    
    for img_name, img_geom in names, geom_all:
        
        json_geom = img_geom[img_name]
        json_str = json.dumps(json_geom)

        return json_str
    
#     ogr_geom = ogr.CreateGeometryFromJson(json_str)
#     geom_ogr = wkb_loads(ogr_geom.ExportToWkb())
#     geom_ogr_all.append(geom)
#     print(geom_ogr)

In [ ]:
teste = rasterio_to_shapely(names, geom_all)

In [ ]:
def rasterio_to_shapely(geom_all):
    geom_ogr_all = []
    
    for image in geom_all:
        
        json_geom = geom_all[i][0]
        json_str = json.dumps(json_geom)
        ogr_geom = ogr.CreateGeometryFromJson(json_str)
        geom_ogr = wkb_loads(ogr_geom.ExportToWkb())
        geom_ogr_all.append(geom_ogr)

    return geom_ogr_all

In [ ]:
def tif_to_union(raster, crs_affine):
    geoms_estradas = []
       
        
#     mask = np.where(raster==1, True, False)
    features_shapes = list(features.shapes(raster, transform = crs_affine))

    for rasterio_shp in features_shapes:
        geoms_estradas.append(rasterio_to_shapely(rasterio_shp))

    union = unary_union(geoms_estradas)

    return union

In [ ]:
def tiffs_to_geoms(rasters, crs_affine_all):
    geoms_estradas = []
    
    for raster_tiff,img_crs_affine in zip(rasters,crs_affine_all):
        
        geoms_estradas.append(tif_to_union(raster_tiff, img_crs_affine))
        
    return geoms_estradas

In [5]:
def rasterio_to_shapely(rasterio_shp):
    
    json_dict = rasterio_shp[0]
    json_str = json.dumps(json_dict)
    ogr_geom = ogr.CreateGeometryFromJson(json_str)
    geom = wkb_loads(ogr_geom.ExportToWkb())

    return geom

def tif_to_union(raster, img_crs_affine):
    geoms_estradas = []
       
        
#     mask = np.where(raster==1, True, False)
    features_shapes = list(features.shapes(raster, transform = img_crs_affine))

    for rasterio_shp in features_shapes:
        geoms_estradas.append(rasterio_to_shapely(rasterio_shp))

    union = unary_union(geoms_estradas)

    return union

def tiffs_to_geoms(rasters, crs_affine_all):
    geoms_estradas = []
    
    for raster_tiff,img_crs_affine in zip(rasters,crs_affine_all):
        
        geoms_estradas.append(tif_to_union(raster_tiff, img_crs_affine))
        
    return geoms_estradas

In [ ]:
geoms_estradas = tiffs_to_geoms(estradas_dilate3, crs_affine_all)

In [ ]:
def export_pol_to_shapefile(unions):
    
    driver_pol = ogr.GetDriverByName("ESRI Shapefile")
    data_source_pol = driver_pol.CreateDataSource("mapbiomas_polygon.shp")
    
    source_pol = osr.SpatialReference()
    source_pol.ImportFromEPSG(32722)

    target_pol = osr.SpatialReference()
    target_pol.ImportFromEPSG(31982)
    
    ogr_transform_pol = osr.CoordinateTransformation(source_pol, target_pol)


    layer = data_source_pol.CreateLayer("geom_polygon", target_pol, ogr.wkbMultiPolygon)


    field_id_estr = ogr.FieldDefn("id_estr", ogr.OFTInteger)
    layer.CreateField(field_id_estr)
    
    field_bloco_estr = ogr.FieldDefn("bloco",ogr.OFTString)
    field_bloco_estr.SetWidth(20)
    layer.CreateField(field_bloco_estr)

    field_area_estr = ogr.FieldDefn("area", ogr.OFTReal)
    layer.CreateField(field_area_estr)
    
    for count, (union_image, nome_bloco) in enumerate(zip(unions, id_images['bloco_id_intersect'])):
        

        geoms_estradas_wkb = union_image.wkb
        ogr_geoms_estradas = ogr.CreateGeometryFromWkb(geoms_estradas_wkb)
        ogr_geoms_estradas.Transform(ogr_transform_pol)

        area = ogr_geoms_estradas.Area()

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("id_estr", count+1)
        feature.SetField("bloco", nome_bloco)
        feature.SetField("area", area)
        feature.SetGeometry(ogr_geoms_estradas)
        layer.CreateFeature(feature)
    
    
    return layer.SyncToDisk()
